In [6]:
# Name: Sai Anish Garapati
# UIN: 650208577

In [7]:
import torch
import pandas as pd
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
import shutil
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image

In [8]:
class TrafficImageDatset(Dataset):
	def __init__(self, csv_file, root_dir, transform=None):
		self.csv_content = pd.read_csv(csv_file, usecols=['ClassId', 'Path'])
		self.root_dir = root_dir
		self.transform = transform

	def __len__(self):
		return len(self.csv_content)

	def __getitem__(self, index):
		img = Image.open(self.root_dir + self.csv_content.iloc[index, 1])
		label = torch.tensor(int(self.csv_content.iloc[index, 0]))

		if self.transform:
			img_tensor = self.transform(img)

		return (img_tensor, label)

In [44]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(2304, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 43)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [41]:
def test_model(device, model, test_loader):
    model.eval()
    corrects = 0

    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            predictions = output.argmax(dim=1, keepdim=True)
            corrects += predictions.eq(label.view_as(predictions)).sum().item()

        print('Correct: {}, Total: {}, Test Accuracy: {}'.format(corrects, len(test_loader), 100.0 * corrects / len(test_loader)))

In [42]:
torch.manual_seed(2021)

loaded_model = Net()
loaded_model.load_state_dict(torch.load('./ML_Model.pt'))

root_path = '../../../../ML_Project_Data/'
test_batch_size=1

transform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_dataset = TrafficImageDatset(
    csv_file=root_path + 'Test.csv', root_dir=root_path, transform=transform)

test_loader = DataLoader(
    dataset=test_dataset, batch_size=test_batch_size, shuffle=False)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

test_model(device, loaded_model, test_loader)

Correct: 12128, Total: 12630, Test Accuracy: 96.02533650039588
